In [2]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import scipy.optimize as optimize
from scipy.optimize import brenth

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

pi=4*np.arctan(1.)

In [3]:
def U(x): return 0.25*x**4-2*alpha*x**3/3.-0.5*x**2
def dU(x): return x**3-2*alpha*x**2-x
def ddU(x): return 3*x**2-4*alpha*x-1.

def T(x): return 1-(dU(x)/g1)**2
def dT(x): return -2*dU(x)*ddU(x)/g1**2

def Ueff(x): return U(x)+T(x)/g2
def dUeff(x): return dU(x)+dT(x)/g2
def ddUeff(x):
    return ddU(x)*(1.-2*ddU(x)/(g2*g1**2))

def integrand(x):
    return dUeff(x)/T(x)

In [4]:
def root(F, x1, x2, div, it):
    X=np.linspace(x1,x2, div)
    roots=np.array([])
    for i in range(div-1):
        if F(X[i])*F(X[i+1])<=0:
            p1=X[i]
            p2=X[i+1]
            r=brenth(F, p1, p2, maxiter=it)
            roots=np.append(roots, r)
    return roots

In [5]:
def integral2(F, X):
    l=len(X)
    Func=np.zeros(l)
    id0=int(0.5*l)
    xmin=X[id0]
    Func[id0]=0
    for i in range(1,l):
        if i<=id0:
            x0 = X[id0-i+1]
            x1 = X[id0-i]
            Func[id0-i]=integrate.quad(F, x0, x1)[0]
            Func[id0-i]=Func[id0-i]+Func[id0-i+1]
        else:
            x0 = X[i-1]
            x1 = X[i]
            Func[i]=integrate.quad(F, x0, x1)[0]
            Func[i]=Func[i]+Func[i-1]
    return Func

In [6]:
def evol(x, h):
    e=np.random.normal(0, 1)
    y=x-h*dU(x)/g1+np.sqrt(2*T(x)*h/(g1*g2))*e
    return y

In [7]:
alpha=0.05
g1=1
g2=0.1

In [8]:
cpU=np.array([alpha-np.sqrt(alpha**2+1),0 ,alpha+np.sqrt(alpha**2+1)]) #this stores all the critical points for functions
cpdU=np.array([2*alpha/3.-np.sqrt(4*alpha**2+3)/3, 2*alpha/3.+np.sqrt(4*alpha**2+3)/3])
cpT=np.sort( np.concatenate([cpU, cpdU] ) )
cpUeff=root(dUeff, -10, 10, 10000, 100)

## $g_1$ mínimo

In [9]:
g1=1.1; step=1.0; error=1e3; T0=min(T(cpT)); it=1; vec=np.array([T0]) #esto debería ser función pero so far no funciona
while error > 1e-4:
    g1=g1-np.sign(T0)*step
    T1=min(T(cpT))
    error=np.absolute(T1)
    vec=np.append(vec, T1)
    dvec=np.gradient(vec)
    if dvec[-1]<1e-2: step=step/2.
    T0=T1
    it+=1
    if it==100: break
g1min=g1

In [10]:
alphas=np.array([0.01, 0.5])

In [14]:
alpha=alphas[0]
xc1 = alpha+np.sqrt(alpha**2+1)
xc2 = alpha-np.sqrt(alpha**2+1)

## Langevin 

In [15]:
cpU=np.array([alpha-np.sqrt(alpha**2+1),0 ,alpha+np.sqrt(alpha**2+1)]) #this stores all the critical points for functions
cpdU=np.array([2*alpha/3.-np.sqrt(4*alpha**2+3)/3, 2*alpha/3.+np.sqrt(4*alpha**2+3)/3])
cpT=np.sort( np.concatenate([cpU, cpdU] ) )
    
g1=1.1; step=1.0; error=1e3; T0=min(T(cpT)); it=1; vec=np.array([T0]) #esto debería ser función pero so far no funciona
while error > 1e-4:
    g1=g1-np.sign(T0)*step
    T1=min(T(cpT))
    error=np.absolute(T1)
    vec=np.append(vec, T1)
    dvec=np.gradient(vec)
    if dvec[-1]<1e-2: step=step/2.
    T0=T1
    it+=1
    if it==100: break
g1min=g1

G1=np.array([1.01*g1min, 2*g1min, 10*g1min])
value=G1/g1min

In [16]:
g1=G1[1]
  
g2=1e-5
step=5
it=1
for i in range(10000):
    aux=ddUeff(cpU[0])
    if aux>0: g2=g2-step
    else: g2=g2+step
    if np.mod(it,100)==0: step=step/20.
    it=it+1
g2c1=g2

g2=1e-5
step=5
it=1
for i in range(10000):
    aux=ddUeff(cpU[2])
    if aux>0: g2=g2-step
    else: g2=g2+step
    if np.mod(it,100)==0: step=step/20.
    it=it+1
g2c2=g2

G2=np.array([0.5*g2c1, g2c1, 0.5*(g2c1+g2c2), g2c2, 2*g2c2, 10*g2c2])

In [21]:
H = np.array([5e-4, 1e-3, 1e-3, 1e-3, 1e-2, 5e-1])
for j in range(len(G2)):
    g2=G2[j]
    h=5e-1
    xmin, xmax=root(T, 99*min(cpT), 99*max(cpT), 500, 100)
    for k in range(20):
        i=0
        x=np.array([xc2])
        while np.abs(x[i])>1e-5:
            y = evol(x[i],h)
            if y > xmax or y < xmin: y=x[i]
            x = np.append(x, y)
            i=i+1
        print(i)

216620
206914


KeyboardInterrupt: 